<a href="https://colab.research.google.com/github/hiroalchem/data_science_lecture_2023/blob/main/Day5_20230309_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 今日の取り組み   
今日はYOLOv5を用いた基本的な物体検出を行ってみます   
今回は[こちら](https://public.roboflow.com/object-detection/bccd)のpublicなdatasetを使用します


In [ ]:
!curl -L "https://public.roboflow.com/ds/9iyNSXKJyb?key=PdEmmZM9Mz" > day5.zip; unzip -d day5 day5.zip; rm day5.zip

In [ ]:
%cd day5
!ls

# 0. 準備
## 0-1: 必要なデータのダウンロード


In [ ]:
%cd /content
!curl -L "https://public.roboflow.com/ds/9iyNSXKJyb?key=PdEmmZM9Mz" > Day5.zip; unzip -d Day5 Day5.zip; rm Day5.zip
%cd Day5

train, valid, testそれぞれのフォルダとdata.yamlが今回使用するものです   
Day5    
&emsp;  |- train   
&emsp;  |- valid   
&emsp;  |- test   
&emsp;  |- data.yaml

## 0-2: 必要なライブラリをインストールし、インポートします。
 

In [ ]:
# /content　に移動します
%cd /content

In [ ]:
# YOLOv5のリポジトリをcloneします。
!git clone https://github.com/ultralytics/yolov5

# リポジトリの中に移動
%cd yolov5

In [ ]:
# 必要なライブラリをrequirements.txtを参照してインストールします。
# requirements.txtとは、リポジトリで必要なライブラリをまとめておいてくれているテキストファイルです。
%pip install -qr requirements.txt

# 必要なライブラリをimportします。

import torch
import os
from pathlib import Path
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# 1. データと設定ファイルの確認

1-1: まず画像とアノテーションを確認します

In [ ]:
# 学習用フォルダ内の画像フォルダとラベルフォルダをpathlib形式で取得しておきます
image_dir = Path('/content/Day5/train/images')
label_dir = Path('/content/Day5/train/labels')

# 画像パスをリストで取得しておきます
image_path_list = sorted(list(image_dir.glob('./*.jpg')))

#最初の5ファイルを出力
print(image_path_list[:5])

In [ ]:
# いくつか必要な関数を作っておきます

# xywhをxyxyに変換する関数
def xywh2xyxy(xywh):
  x1 = xywh[0] - xywh[2]/2
  y1 = xywh[1] - xywh[3]/2
  x2 = x1 + xywh[2]
  y2 = y1 + xywh[3]
  return x1, y1, x2, y2

# 画像パスとラベルフォルダのパスを渡すとboxを描画して表示してくれる関数
def draw_box(image_path, label_dir):
  # クラスごとに色を割り当てるためのリストを作っておきます
  colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]
  # ラベルのパスの取得
  label_path = label_dir / image_path.with_suffix('.txt').name

  # 画像を読み込んでサイズを取得しておきます
  im = Image.open(image_path)
  width, height = im.size

  # 描画するオブジェクトの生成
  draw = ImageDraw.Draw(im)

  # テキストファイルを1行ずつ読み込む
  with open(label_path, 'r') as f:
      for line in f.readlines():
        label, x, y, w, h = line.split(' ')
        # 端点に変換
        x1, y1, x2, y2 = xywh2xyxy((float(x), float(y), float(w), float(h)))
        # 矩形を書き込みます
        # 座標は0~1になっているので、画像の縦横サイズをかけるのを忘れずに
        # labelの値から色を割り当てます
        draw.rectangle((x1*width, y1*height, x2*width, y2*height), outline=colors[int(label)], width=3)

  # 全ての行を読んだら表示
  plt.imshow(im)


In [ ]:
# 画像を確認してみます
draw_box(image_path_list[2], label_dir)

In [ ]:
# 学習をします
# 画像サイズは 320, バッチサイズ 2, Day5内のdata.yamlを参照, yolov5sを学習, エポックは50にしてみます。
!python train.py --img 320 --batch 2 --epochs 50 --data /content/Day5/data.yaml --weights yolov5s.pt --cache

In [ ]:
# val.pyで評価ができます。
# runs/train/にexpフォルダができています。その中で最高の性能であるbest.ptを選択します。

!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 320 --data /content/Day5/data.yaml

In [ ]:
# --task test とすることでテストデータも評価できます。
!python val.py --task test --weights /content/yolov5/runs/train/exp/weights/best.pt --img 320 --data /content/Day5/data.yaml

In [ ]:
# 検出結果を保存するにはdetect.pyを使用します。
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 320 --source  /content/Day5/test/images 

In [ ]:
# 検出結果の画像を確認してみましょう。
import glob
from IPython.display import Image as iImage
from IPython.display import display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'):
    display(iImage(filename=imageName))
    print("\n")